In [47]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import StructField, StructType, IntegerType , StringType ,FloatType, Row
from pyspark.storagelevel import StorageLevel #Craer niveles de percistencia de datos

In [61]:
sc = SparkContext(master='local',appName='prac') 
sql_context = SQLContext(sc)

In [2]:
sc = (SparkSession.builder
                  .appName('practica')
                  .master('local[5]')
                  .getOrCreate()
     )
sql_context = SQLContext(sc)

In [3]:
!head -n 5 /home/cesar/Documents/prac_sp/curso-apache-spark-platzi/files/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis
4,1906 Verano,1906,Verano,Athina


In [62]:
path = '/home/cesar/Documents/prac_sp/curso-apache-spark-platzi/files/'

#Creo un schema para poder cargar el archivo  
juegoSchema = (StructType
               ([
    StructField('id',IntegerType(),False), #El tercer atributo hace referencia a si es obligatorio que exista el dato
    StructField('name_game',StringType(),False),
    StructField('year',IntegerType(),False),
    StructField('season',StringType(),False),
    StructField('city',StringType(),False)
])
              )

#Cargo el archivo como una base de datos 

juegos_df = sql_context.read.schema(juegoSchema).option('header','true').csv(path+'juegos.csv')

In [63]:
juegos_df.show(2)

+---+-----------+----+------+------+
| id|  name_game|year|season|  city|
+---+-----------+----+------+------+
|  1|1896 Verano|1896|Verano|Athina|
|  2|1900 Verano|1900|Verano| Paris|
+---+-----------+----+------+------+
only showing top 2 rows



# Percistencia

In [64]:
# Ver si esta almacenado 
juegos_df.is_cached

False

In [65]:
# Almacenar en cache. que es igual a storageLevel only memory 
juegos_df.rdd.cache()

MapPartitionsRDD[8] at javaToPython at NativeMethodAccessorImpl.java:0

In [66]:
juegos_df.is_cached

False

In [67]:
#Ver el tipo de persistencia 
juegos_df.rdd.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [68]:
#Quitarle la persistencia 
juegos_df.rdd.unpersist()

MapPartitionsRDD[8] at javaToPython at NativeMethodAccessorImpl.java:0

## Storage level
**class pyspark.StorageLevel(useDisk, useMemory, useOffHeap, deserialized, replication=1)**

**NOTA:** Si quiero hacer un persist, y ya esta en caché, me va a dar un error, para eso lo tengo que bajar de momoria para hacer la persistencia, eso se hace con rdd.unpersist()

In [69]:
juegos_df.rdd.persist(StorageLevel.MEMORY_AND_DISK_2)

MapPartitionsRDD[8] at javaToPython at NativeMethodAccessorImpl.java:0

Crear un storage level personalizado

In [70]:
StorageLevel.MEMORY_AND_DISK_3 = StorageLevel(True,True,False,False,3)

In [71]:
juegos_df.rdd.unpersist()

MapPartitionsRDD[8] at javaToPython at NativeMethodAccessorImpl.java:0

In [72]:
juegos_df.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

MapPartitionsRDD[8] at javaToPython at NativeMethodAccessorImpl.java:0

# Particiones

En esta salida me va a aparecer 5 particiones, porque a la hoa de cargar el sc, en el master coloqué 'local[5]' lo que quiere decir que va a particionar los datos en 5

In [74]:
df = sc.rdd.range(0,20)
df.rdd.getNumPartitions()

AttributeError: 'SparkContext' object has no attribute 'rdd'

In [ ]:
juegos_df.rdd.getNumPartitions()

In [13]:
#También puedo setear las particiones manualmente
paises = sc.sparkContext.textFile(path+'paises.csv',10)
paises.take(3)

['id,equipo,sigla', '1,30. Februar,AUT', '2,A North American Team,MEX']

In [14]:
paises.getNumPartitions()

10

In [17]:
paises.saveAsTextFile('./archivos_salida/')

Me va a crear una carpeta donde va a guardar los datos particionados 

In [18]:
%ls archivos_salida/

part-00000  part-00002  part-00004  part-00006  part-00008  _SUCCESS
part-00001  part-00003  part-00005  part-00007  part-00009


In [19]:
!head -n 5 archivos_salida/part-00000

id,equipo,sigla
1,30. Februar,AUT
2,A North American Team,MEX
3,Acipactli,MEX
4,Acturus,ARG


In [20]:
!head -n 5 archivos_salida/part-00001

132,Boreas-2,NED
133,Bosnia and Herzegovina,BIH
134,Boston Archers,USA
135,Botswana,BOT
136,Brandenburg,EST


### Cargar los archivos particionados 

In [21]:
rdd = sc.sparkContext.wholeTextFiles('archivos_salida/*')

In [22]:
#Esta salida va estar sucia
rdd.take(1)

[('file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00005',
  '618,Malaya,MAL\n619,Malaysia,MAS\n620,Malaysia-1,MAS\n621,Malaysia-2,MAS\n622,Maldives,MDV\n623,Mali,MLI\n624,Malta,MLT\n625,Mamie-4,FRA\n626,Mamie-6,FRA\n627,Manana,CAN\n628,Marga-2,SWE\n629,Margaret,USA\n630,Mari,SWE\n631,Marie-Tim,GRE\n632,"Marinai della nave da guerra ""Varese""",ITA\n633,Marmara,TUR\n634,Marmi II-1,NOR\n635,Marshall Islands,MHL\n636,Marsouin-26,FRA\n637,Marsouin-8,FRA\n638,Martha-1,FRA\n639,Martha-27,FRA\n640,Maryke,AUS\n641,Mascaret-28,FRA\n642,Mascaret-4,FRA\n643,Mascotte,NED\n644,Matrero II,ARG\n645,Mauritania,MTN\n646,Mauritius,MRI\n647,Mauritius-1,MRI\n648,Mauritius-2,MRI\n649,May Be,SWE\n650,May Be VII,SWE\n651,May-Be 1960,AUT\n652,Mebi,ITA\n653,Merope,ITA\n654,Merope III,ITA\n655,Meteor,BEL\n656,Mexico,MEX\n657,Mexico-1,MEX\n658,Mexico-2,MEX\n659,Mignon-29,FRA\n660,Mignon-3,FRA\n661,Milwaukee Athletic Club-1,USA\n662,"Milwaukee Turnverein; Milwaukee",USA\n663,Minerva Amsterd

In [23]:
lista = rdd.mapValues(lambda x : x.split()).collect()

In [24]:
lista

[('file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00005',
  ['618,Malaya,MAL',
   '619,Malaysia,MAS',
   '620,Malaysia-1,MAS',
   '621,Malaysia-2,MAS',
   '622,Maldives,MDV',
   '623,Mali,MLI',
   '624,Malta,MLT',
   '625,Mamie-4,FRA',
   '626,Mamie-6,FRA',
   '627,Manana,CAN',
   '628,Marga-2,SWE',
   '629,Margaret,USA',
   '630,Mari,SWE',
   '631,Marie-Tim,GRE',
   '632,"Marinai',
   'della',
   'nave',
   'da',
   'guerra',
   '""Varese""",ITA',
   '633,Marmara,TUR',
   '634,Marmi',
   'II-1,NOR',
   '635,Marshall',
   'Islands,MHL',
   '636,Marsouin-26,FRA',
   '637,Marsouin-8,FRA',
   '638,Martha-1,FRA',
   '639,Martha-27,FRA',
   '640,Maryke,AUS',
   '641,Mascaret-28,FRA',
   '642,Mascaret-4,FRA',
   '643,Mascotte,NED',
   '644,Matrero',
   'II,ARG',
   '645,Mauritania,MTN',
   '646,Mauritius,MRI',
   '647,Mauritius-1,MRI',
   '648,Mauritius-2,MRI',
   '649,May',
   'Be,SWE',
   '650,May',
   'Be',
   'VII,SWE',
   '651,May-Be',
   '1960,AUT',
   '652,Mebi,

Optengo los archivos de las particiones , y ordeno para que la tabla este en orden según como fue particionada

In [25]:
lista = [i[0] for i in lista]
lista.sort()
lista

['file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00000',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00001',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00002',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00003',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00004',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00005',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00006',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00007',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00008',
 'file:/home/cesar/Documents/practica_cursos/spark/archivos_salida/part-00009']

Ahora lo que tengo que hacer en colocar todas las rutas como una sola cadena separada por ,

In [26]:
rdd_clean = sc.sparkContext.textFile(','.join(lista),10)

En este punto ya tengo el rdd limpio, pero me queda todo en un solo bloque, por lo que tengo que separar otra vez por comar para poder crear las columnas 

In [27]:
rdd_clean.take(4)

['id,equipo,sigla',
 '1,30. Februar,AUT',
 '2,A North American Team,MEX',
 '3,Acipactli,MEX']

In [28]:
rdd_clean = rdd_clean.map(lambda x: x.split('v'))

In [29]:
rdd_clean.take(20)

[['id,equipo,sigla'],
 ['1,30. Februar,AUT'],
 ['2,A North American Team,MEX'],
 ['3,Acipactli,MEX'],
 ['4,Acturus,ARG'],
 ['5,Afghanistan,AFG'],
 ['6,Akatonbo,IRL'],
 ['7,Alain IV,SUI'],
 ['8,Albania,ALB'],
 ['9,Alcaid,POR'],
 ['10,Alcyon-6,FRA'],
 ['11,Alcyon-7,FRA'],
 ['12,Aldebaran,ITA'],
 ['13,Aldebaran II,ITA'],
 ['14,Aletta,IRL'],
 ['15,Algeria,ALG'],
 ['16,Ali-Baba II,SWE'],
 ['17,Ali-Baba IV,SUI'],
 ['18,Ali-Baba IX,SUI'],
 ['19,Ali-Baba VI,SUI']]

In [30]:
rdd_clean.sample(False,0.5).take(5)

[['id,equipo,sigla'],
 ['2,A North American Team,MEX'],
 ['3,Acipactli,MEX'],
 ['9,Alcaid,POR'],
 ['16,Ali-Baba II,SWE']]

In [31]:
rdd_clean.count()

1185

In [32]:
rdd_clean.sample(50,2)

PythonRDD[28] at RDD at PythonRDD.scala:53

In [33]:
sc

In [60]:
sc.stop()